# Comparaison EOS monophasique

In [ ]:
from trustutils import run

run.introduction("Yannick Gorsse")

run.TRUST_parameters()

### Description: 



In [ ]:
import os
from string import Template

run.reset()
run.initBuildDirectory()
origin, build_dir = os.getcwd(), run.BUILD_DIRECTORY

fluids_template = Template("liquide_eau Fluide_generique_$lib { model $model fluid $liq }")

models = {
    "EOS": {"liq": "waterliquid", "vap": "watervapor", "sat": "waterliquid", "models": ["cathare2", "refprop10"], "symbol" : "o"},
    "CoolProp": {
        "liq": "water phase liquid",
        "vap": "water phase vapor",
        "sat": "water phase liquid",
        "models": ["BICUBIC&HEOS", "TTSE&HEOS", "refprop"],
        "symbol" : "-"
    },
}

for lib, dict_mods in models.items():
    for model in dict_mods["models"]:
        fluids = fluids_template.substitute({"lib": lib, "model": model, "liq": dict_mods["liq"], "vap": dict_mods["vap"], "sat": dict_mods["sat"]})
        run.addCaseFromTemplate("jdd.data", f"{lib}/{model.replace('&', '_')}", {"fluids": fluids})

run.runCases()
run.tablePerf()


In [ ]:
from trustutils import plot

a = plot.Graph(f"Residus")
for lib, dict_mods in models.items():
    for model in dict_mods["models"]:
        a.addResidu(f"{build_dir}/{lib}/{model.replace('&', '_')}/jdd.dt_ev", label=f"{lib} - {model}")

a.scale(yscale='log')

In [ ]:
from trustutils import plot

vars = ("V", "P", "RHO", "EINT")

for i, var in enumerate(vars):
    a = plot.Graph(var)
    for lib, dict_mods in models.items():
        for model in dict_mods["models"]:
            a.addSegment(f"{build_dir}/{lib}/{model.replace('&', '_')}/jdd_{var}.son", label=f"{lib} - {model}", marker=dict_mods["symbol"])
